# 가중치 그래프

## 11.1 가중치 그래프란?
가중치 그래프(weighted graph): 간선에 비용이나 가중치가 할당된 그래프  
가중치 그래프는 수학적으로 G = (V,E,w)와 같이 표현한다.
- V(G)는 그래프 G의 정점들의 집합
- E(G)는 그래프 G의 간선들의 집합
- w(e)는 간선 e의 강도로 비용 또는 길이

## 11.2 가중치 그래프의 표현

### 인접 행렬을 이용한 표현

In [1]:
vertex = ['A','B','C','D','E','F','G']
weight = [[None, 29, None, None, None, 10, None],
         [29, None, 16, None, None, None, 15],
         [None, 16, None, 12, None, None, None],
         [None, None, 12, None, 22, None, 18],
         [None, None, None, 22, None, 27, 25],
         [10, None, None, None, 27, None, None],
         [None, 15, None, 18, 25, None, None]]
graph = (vertex, weight)

In [2]:
def weightSum(vlist, W):
    sum = 0
    for i in range(len(vlist)):
        for j in range(i+1, len(vlist)):
            if W[i][j] != None:
                sum += W[i][j]
    return sum
print('AM : weight sum = ', weightSum(vertex, weight))

AM : weight sum =  174


In [4]:
def printAllEdges(vlist, W):
    for i in range(len(vlist)):
        for j in range(i+1, len(W[i])):
            if W[i][j] != None and W[i][j] != 0:
                print("(%s,%s,%d)"%(vlist[i], vlist[j],W[i][j]), end = ' ')
    print()
printAllEdges(vertex, weight)

(A,B,29) (A,F,10) (B,C,16) (B,G,15) (C,D,12) (D,E,22) (D,G,18) (E,F,27) (E,G,25) 


### 인접 리스트를 이용한 표현

In [5]:
graphA = {'A' : {('B',29),('F',10)},
         'B' : {('A',29), ('C',16), ('G',15)},
         'C' : {('B',16), ('D',12)},
         'D' : {('C',12), ('E',22), ('G',18)},
         'E' : {('D',22), ('F', 27), ('G',25)},
         'F' : {('A',10), ('E',27)},
         'G' : {('B',15), ('D',18), ('E',25)}}

In [6]:
def weightSum(graph):
    sum = 0
    for v in graph:
        for e in graph[v]:
            sum += e[1]
    return sum//2

def printAllEdges(graph):
    for v in graph:
        for e in graph[v]:
            print('(%s,%s,%d)'%(v,e[0],e[1]), end = ' ')

print('AL : weigth sum = ', weightSum(graphA))
printAllEdges(graphA)

AL : weigth sum =  174
(A,B,29) (A,F,10) (B,G,15) (B,A,29) (B,C,16) (C,D,12) (C,B,16) (D,G,18) (D,C,12) (D,E,22) (E,D,22) (E,G,25) (E,F,27) (F,E,27) (F,A,10) (G,B,15) (G,E,25) (G,D,18) 

## 11.3 최소비용 신장 트리
- 그래프의 모든 정점들은 연결되어야 한다.
- 연결에 필요한 간선의 가중치 합(비용)이 최소가 되어야 한다.
- 사이클은 두 정점을 연결하는 두 가지 경로를 제공하므로 비용 측면에서 바람직하지 않다. 따라서 사이클이 없이 (n-1)개의 간선만을 사용해야 한다.

### Kruskal의 MST 알고리즘
Kruskal 의 알고리즘은 탐욕적인 방법(greedy method)를 사용한다. 각 단계에서 사이클을 이루지 않는 최소 비용 간선을 선택한다. 이러한 과정을 반복하여 그래프의 모든 정점을 최소비용으로 연결하는 최적 해답을 구한다.  
<br>
Kruskal의 최소 비용 신장 트리 알고리즘
1. 그래프의 모든 간선을 가중치에 따라 오름차순으로 정렬한다.
2. 가장 가중치가 작은 간선 e를 뽑는다.
3. e를 신장 트리에 넣었을 때 사이클이 생기면 넣지 않고 2번으로 이동한다.
4. 사이클이 생기지 않으면 최소 신장 트리에 삽입한다.
5. n-1개의 간선이 삽입될 때 까지 2번으로 이동한다. (n은 정점의 개수)

#### union과 find 연산

In [7]:
# 그래프의 모든 정점에 대해 부모 노드의 인덱스를 저장하기 위해 전역변수로 사용
parent = []
set_size = 0

def init_set(nSets):
    global set_size, parent
    set_size = nSets
    for i in range(nSets):
        parent.append(-1)
        
def find(id):
    while (parent[id] >= 0):
        id = parent[id]
    return id

def union(s1, s2):
    global set_size
    parent[s1] = s2
    set_size = set_size -1

In [8]:
#kruskal의 최소 비용 신장 트리 프로그램
def MSTkruskal(vertex, adj):
    vsize = len(vertex)
    init_set(vsize)
    eList = []
    
    for i in range(vsize - 1): #모든 간선을 리스트에 넣음
        for j in range(i+1, vsize):
            if adj[i][j] != None:
                eList.append((i,j,adj[i][j]))
                
    #간선 리스트를 가중치의 내림차순으로 정렬
    eList.sort(key = lambda e: e[2], reverse= True)
    
    edgeAccepted = 0
    while edgeAccepted < vsize - 1:
        e = eList.pop(-1) #가장 작은 가중치를 가진 간선
        uset = find(e[0]) #두 정점이 속한 집합 번호
        vset = find(e[1])
        
        if uset != vset:
            print('간선 추가: (%s,%s,%d)'%(vertex[e[0]], vertex[e[1]], e[2]))
            union(uset, vset)
            edgeAccepted += 1

In [9]:
vertex = ['A','B','C','D','E','F','G']
weight = [[None, 29, None, None, None, 10, None],
         [29, None, 16, None, None, None, 15],
         [None, 16, None, 12, None, None, None],
         [None, None, 12, None, 22, None, 18],
         [None, None, None, 22, None, 27, 25],
         [10, None, None, None, 27, None, None],
         [None, 15, None, 18, 25, None, None]]

print("MST By Kruskal's Algorithm")
MSTkruskal(vertex, weight)

MST By Kruskal's Algorithm
간선 추가: (A,F,10)
간선 추가: (C,D,12)
간선 추가: (B,G,15)
간선 추가: (B,C,16)
간선 추가: (D,E,22)
간선 추가: (E,F,27)


Kruskal의 알고리즘의 시간 복잡도는 간선들을 정렬하는 시간에 좌우된다. 따라서 퀵 정렬이나 최소 힙와 같은 효율적인 정렬 알고리즘을 사용한다면 시간 복잡도는 $elog_{2}e$이다. (e는 간선의 개수)

### Prim의 MST 알고리즘
1. 그래프에서 시작 정점을 선택하여 초기 트리를 만든다.
2. 현재 트리의 정점들과 인접한 정점들 중에서 간선의 가중치가 가장 작은 정점 v를 선택한다.
3. 이 정점 v와 이때의 간선을 트리에 추가한다.
4. 모든 정점이 삽입될 때 까지 2번으로 이동한다.

In [10]:
INF = 9999
#현재 트리에 인접한 정점들 중에서 가장 가까운 정점을 찾는 함수
def getMinVertex(dist, selected):
    minv = 0
    mindist = INF
    for v in range(len(dist)):
        if not selected[v] and dist[v] < mindist:
            mindist = dist[v]
            minv = v
    return minv

In [11]:
#Prim의 최소 비용 신장 트리 프로그램
def MSTPrim(vertex, adj):
    vsize = len(vertex)
    dist = [INF]*vsize
    selected = [False]*vsize
    dist[0] = 0
    
    for i in range(vsize):
        u = getMinVertex(dist, selected)
        selected[u] = True
        print(vertex[u], end = ' ')
        for v in range(visze):
            if (adj[u][v] != None):
                if selected[v] == False and adj[u][v] < dist[v]:
                    dist[v] = adj[u][v]
                    
    print()

Prim 알고리즘의 시간 복잡도는 $O(n^2)$이다.  
간선의 개수가 매우 적은 희박한 그래프(sparse graph)를 대상으로 할 경우에는 Kruskal의 알고리즘이 적합하고, 반대로 완전 그래프와 같이 간선이 매우 많은 그래프의 경우에는 Prim의 알고리즘이 유리하다.

## 11.4 최단 경로
가중치 그래프에서 두 정점을 연결하는 여러 경로들 중에서 간선들의 가중치 합이 최소가 되는 경로를 찾는 문제

### Dijkstra의 최단 경로 알고리즘
하나의 시작 정점 v에서 다른 모든 정점까지의 최단 경로를 찾는 알고리즘

In [1]:
INF = 9999
#최소 dist 정점을 찾는 함수
def choose_vertex(dist, found):
    min = INF
    minpos = -1
    for i in range(len(dist)): #모든 정점 중에서
        if dist[i] < min and found[i] == False: #방문하지 않은 최소 dist 정점
            min = dist[i]
            minpos = i
    return minpos # 최소 dist 정점의 인덱스 반환

def shortest_path_dijkstra(vtx, adj, start):
    vsize = len(vtx) # 정점의 수
    dist = list(adj[start]) #dist 배열 생성 및 초기화
    path = [start] * vsize #path 배열 생성 및 초기화
    found = [False] * vsize #found 배열 생성 및 초기화
    found[start] = True #시작정점: 이미 찾아짐
    dist[start] = 0 #시작정점의 거리: 0
    
    for i in range(vsize):
        print('Step%2d: '%(i+1), dist) #단계별 dist[]출력용
        u = choose_vertex(dist, found) #최소 dist 정점 u 탐색
        found[u] = True
        
        for w in range(vsize): #모든 정점에 대해
            if not found[w]: # 아직 찾아지지 않았으면
                if dist[u] + adj[u][w] < dist[w]: #갱신 조건 검사
                    dist[w] = dist[u] + adj[u][w] #dist 갱신
                    path[w] = u #이전 정점 갱신
                    
    return path #찾아진 최단 경로 반환

In [2]:
vertex = ['A','B','C','D','E','F','G']
weight = [[0, 7, INF, INF, 3, 10, INF],
         [7, 0, 4, 10, 2, 6, INF],
         [INF, 4, 0, 2, INF ,INF ,INF],
         [INF,10,2,0,11,9,4],
         [3, 2, INF,11,0,13,5],
         [10, 6, INF,9,13,0,INF],
         [INF,INF,INF,4,5,INF,0]]

print('Shortest Path By Dijkstra Algorithm')
start = 0
path = shortest_path_dijkstra(vertex, weight, start)

#최종 경로를 출력하기 위한 코드
for end in range(len(vertex)):
    if end != start:
        print('[최단 경로: %s -> %s] %s'%(vertex[start], vertex[end], vertex[end]), end='')
        while path[end] != start:
            print(" <- %s"%vertex[path[end]], end = '')
            end = path[end]
        print(' <- %s'%vertex[path[end]])

Shortest Path By Dijkstra Algorithm
Step 1:  [0, 7, 9999, 9999, 3, 10, 9999]
Step 2:  [0, 5, 9999, 14, 3, 10, 8]
Step 3:  [0, 5, 9, 14, 3, 10, 8]
Step 4:  [0, 5, 9, 12, 3, 10, 8]
Step 5:  [0, 5, 9, 11, 3, 10, 8]
Step 6:  [0, 5, 9, 11, 3, 10, 8]
Step 7:  [0, 5, 9, 11, 3, 10, 8]
[최단 경로: A -> B] B <- E <- A
[최단 경로: A -> C] C <- B <- E <- A
[최단 경로: A -> D] D <- C <- B <- E <- A
[최단 경로: A -> E] E <- A
[최단 경로: A -> F] F <- A
[최단 경로: A -> G] G <- E <- A


최단 경로 알고리즘은 주 반복문을 n번 반복하고 내부 반복문을 2n번 반복하므로 $O(n^2)$의 시간 복잡도를 갖는다.

### Floyd의 최단 경로 알고리즘
그래프의 모든 정점사이의 최단경로를 한꺼번에 찾아준다.

$A^{k}[i][j]$를 0부터 k까지의 정점만을 이용한 정점 i에서 j까지의 최단 경로라고 하자  
$A^{-1}[i][j]$는 인접 행렬  
$A^{-1}[i][j]$ -> $A^{0}[i][j]$ -> $A^{1}[i][j]$ -> ... -> $A^{n-1}[i][j]$  
$A^{k}[i][j]$ <- $min(A^{k-1}[i][j], A^{k-1}[i][k] + A^{k-1}[k][j])$  
이것은 정점 k를 경유하는 것이 보다 좋은 경로이면 $A^{k-1}[i][j]$의 값이 변경되고, 그렇지 않으면 이전 값을 유지한다는 의미이다.

In [4]:
INF = 9999
def printA(A): #현재의 A행렬을 화면에 출력하는 함수
    vsize = len(A)
    print('======================================')
    for i in range(vsize):
        for j in range(vsize):
            if (A[i][j] == INF):
                print('INF', end = '')
            else:
                print("%4d"%A[i][j], end = '')
        print("")

def shortest_path_floyd(vertex, adj):
    vsize = len(vertex) #정점의 개수
    A = list(adj) #2차원 배열의 복사
    for i in range(vsize):
        A[i] = list(adj[i])
    
    for k in range(vsize):
        for i in range(vsize):
            for j in range(vsize):
                if A[i][k] + A[k][j] < A[i][j]:
                    A[i][j] = A[i][k] + A[k][j]
        printA(A)

In [5]:
shortest_path_floyd(vertex, weight)

   0   7INFINF   3  10INF
   7   0   4  10   2   6INF
INF   4   0   2INFINFINF
INF  10   2   0  11   9   4
   3   2INF  11   0  13   5
  10   6INF   9  13   0INF
INFINFINF   4   5INF   0
   0   7  11  17   3  10INF
   7   0   4  10   2   6INF
  11   4   0   2   6  10INF
  17  10   2   0  11   9   4
   3   2   6  11   0   8   5
  10   6  10   9   8   0INF
INFINFINF   4   5INF   0
   0   7  11  13   3  10INF
   7   0   4   6   2   6INF
  11   4   0   2   6  10INF
  13   6   2   0   8   9   4
   3   2   6   8   0   8   5
  10   6  10   9   8   0INF
INFINFINF   4   5INF   0
   0   7  11  13   3  10  17
   7   0   4   6   2   6  10
  11   4   0   2   6  10   6
  13   6   2   0   8   9   4
   3   2   6   8   0   8   5
  10   6  10   9   8   0  13
  17  10   6   4   5  13   0
   0   5   9  11   3  10   8
   5   0   4   6   2   6   7
   9   4   0   2   6  10   6
  11   6   2   0   8   9   4
   3   2   6   8   0   8   5
  10   6  10   9   8   0  13
   8   7   6   4   5  13   0
   0   5   9  11 

Floyd 알고리즘의 시간 복잡도는 $O(n^3)$이다